In [1]:
import pandas as pd
import numpy as np

In [3]:
import statistics as st
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [5]:
# Load dataset
df = pd.read_csv('/content/heart.csv')

In [6]:
# spliting the dataset
X = df.drop(columns='target', axis=1)
y = df['target']

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

**Create Function**

In [8]:
from sklearn.model_selection import StratifiedKFold

In [10]:
def Stacking(model, train, y, test, n_fold):

  folds = StratifiedKFold(n_fold, shuffle=True, random_state= 1)
  test_pred = np.zeros((test.shape[0],))
  train_pred = np.zeros((train.shape[0],))

  for train_indices, val_indices in folds.split(train, y.values):
    x_train, x_val = train.iloc[train_indices], train.iloc[val_indices]
    y_train, y_val = y.iloc[train_indices], y.iloc[val_indices]

    model.fit(x_train, y_train)

    train_pred[val_indices] = model.predict(x_val)
    test_pred = test_pred + model.predict(test)

  test_pred = test_pred / n_fold
  return test_pred.reshape(-1, 1), train_pred.reshape(-1, 1)

### **Now Create Two Base Models**

**Decision Tree**

In [11]:
model1 = DecisionTreeClassifier()

In [13]:
test_pred1, train_pred1 = Stacking(model=model1, n_fold=10, train=x_train, test= x_test, y= y_train)

In [14]:
train_pred1 = pd.DataFrame(train_pred1)

In [15]:
test_pred1 = pd.DataFrame(test_pred1)

**K - Nearest Neighbors**

In [16]:
model2 = KNeighborsClassifier()

In [17]:
test_pred2, train_pred2 = Stacking(model=model2, n_fold=10, train=x_train, test=x_test, y=y_train)

In [18]:
train_pred2 = pd.DataFrame(train_pred2)

In [19]:
test_pred2 = pd.DataFrame(test_pred2)

**Logistic Regression**

In [20]:
df = np.concatenate([train_pred1, train_pred2], axis=1)

In [21]:
df_test = np.concatenate([test_pred1, test_pred2], axis=1)

In [22]:
model = LogisticRegression(random_state=1)

In [23]:
model.fit(df, y_train)

LogisticRegression(random_state=1)

In [24]:
y_pred = model.predict(df_test)

In [25]:
# Calculate accuracy
from sklearn.metrics import accuracy_score

In [26]:
score = accuracy_score(y_test, y_pred)

In [27]:
print(f"Accuracy: {score}")

Accuracy: 0.8026315789473685
